# **Necssary Downloads**




In [1]:
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 4.1 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-aifpgbi8
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-aifpgbi8
  Resolved https://github.com/deepset-ai/haystack.git to commit 820742cac7c6d8426bb9d465106193cf79944686
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.4/228.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

# **Importing Packages**


In [2]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [3]:
from pprint import pprint
from tqdm import tqdm
from haystack.nodes import QuestionGenerator, BM25Retriever, FARMReader
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.pipelines import (
    QuestionGenerationPipeline,
    RetrieverQuestionGenerationPipeline,
    QuestionAnswerGenerationPipeline,
)
from haystack.utils import launch_es, print_questions
import pandas as pd 
from haystack.pipelines import ExtractiveQAPipeline
from haystack.nodes import TfidfRetriever


In [4]:
launch_es()


In [5]:
# Option 2: In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
! sleep 30

# **Read the KeyPhrase DataFrame**


In [8]:
df = pd.read_csv("/content/drive/MyDrive/KeyPhrases_DataFrame.csv")
paragraphs = []

for x in range(len(df['Paragraphs'])):
  paragraphs.append(df['Phrases'][x] + ' ' + df['Paragraphs'][x])

# **Prepare the Paragraphs**


In [11]:
docs = [
    
]
for paragraph in paragraphs:
  docs.append({"content": paragraph})

In [12]:
docs

[{'content': 'neural network architecture convolutional neural network hidden layers convolutional neural network  convnet  consists convolutional layers , pooling layers , fully connected layers , normalization layers .'},
 {'content': 'deep learning deep learning defined multi neural network architecture containing large number parameters layers .'},
 {'content': 'artificial intelligence types ai  examining ability imitate natural intelligence , hardware use , application real world , artificial intelligence divided three parts .'},
 {'content': 'human brain deep learning architectures classified following parts  artificial neural network  ann   artificial neural network computing system designed stimulate way human brain analyzes processes information .'},
 {'content': 'general intelligence immense challenge achieving strong ai surprising consider human brain model creating general intelligence .'},
 {'content': 'artificial neural network deep learning architectures classified follo

# **Build QA with Elastic Search**


In [13]:
document_store = ElasticsearchDocumentStore()
document_store.write_documents(docs)
question_generator = QuestionGenerator()

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry
INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Using sep_token, but it is not set yet.


# **Haystack QA generation algorithms**


In [21]:
def question_generation_pipeline(question_generator, document_store):
  question_generation_pipeline = QuestionGenerationPipeline(question_generator)
  for idx, document in enumerate(document_store):

    print(f"\n * Generating questions for document {idx}: {document.content[:100]}...\n")
    result = question_generation_pipeline.run(documents=[document])
    print_questions(result)

def question_answer_generation_pipeline(question_generator, document_store):
  reader = FARMReader("deepset/roberta-base-squad2")
  qag_pipeline = QuestionAnswerGenerationPipeline(question_generator, reader)
  for idx, document in enumerate(tqdm(document_store)):

      print(f"\n * Generating questions and answers for document {idx}: {document.content[:100]}...\n")
      result = qag_pipeline.run(documents=[document])
      print_questions(result)

# **Question Generation using haystack**


In [18]:
question_generation_pipeline(question_generator, document_store)


 * Generating questions for document 0: neural network architecture convolutional neural network hidden layers convolutional neural network ...


Generated questions:
 - What is a convolutional neural network convnet composed of?

 * Generating questions for document 1: deep learning deep learning defined multi neural network architecture containing large number parame...


Generated questions:
 - deep learning deep learning defined multi neural network architecture containing large number of parameters layers

 * Generating questions for document 2: artificial intelligence types ai  examining ability imitate natural intelligence , hardware use , ap...


Generated questions:
 - How many parts is artificial intelligence divided into?

 * Generating questions for document 3: human brain deep learning architectures classified following parts  artificial neural network  ann  ...


Generated questions:
 - Human brain deep learning architectures classified as what?
 - Artificial neural netw

# **Question Answer Generation using haystack**


In [22]:
question_answer_generation_pipeline(question_generator, document_store)

INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)


INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.


INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.modeling.infer:Got ya 2 parallel workers to do inference ...
INFO:haystack.modeling.infer: 0     0  
INFO:haystack.modeling.infer:/w\   /w\ 
INFO:haystack.modeling.infer:/'\   / \ 
0it [00:00, ?it/s]


 * Generating questions and answers for document 0: neural network architecture convolutional neural network hidden layers convolutional neural network ...



...


Generated pairs:
 - Q: What does reinforcement learning operate on?
      A: principle  actually , video games
 - Q: Video games common test environment kind research?
      A: reinforcement learning

 * Generating questions and answers for document 33: natural intelligence types ai  examining ability imitate natural intelligence , hardware use , appli...




Inferencing Samples: 100%|██████████| 1/1 [00:01<00:00,  1.43s/ Batches]
34it [02:56,  5.49s/it]


Generated pairs:
 - Q: Natural intelligence types examining ability imitate natural intelligence, hardware use, application real world, artificial intelligence divided three parts?
      A: ai

 * Generating questions and answers for document 34: neural layers filters input data processes neural layers predict classify information ....




Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.04 Batches/s]
35it [03:00,  5.15s/it]


Generated pairs:
 - Q: What filters input data processes neural layers predict classify information?
      A: neural layers


#**Initialize Reader, Retriver and Pipeline**

In [35]:
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)
retriever = TfidfRetriever(document_store=document_store)
pipe = ExtractiveQAPipeline(reader, retriever)

INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)
INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.
INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.modeling.infer:Got ya 2 parallel workers to do inference ...
INFO:haystack.modeling.infer: 0     0  
INFO:haystack.modeling.infer:/w\   /w\ 
INFO:haystack.modeling.infer:/'\   / \ 
INFO:haystack.nodes.retriever.sparse:Found 35 candidate paragraphs from 35 docs in DB


#**Load Our Question and Answer Data Frame that is generated by T5**

In [28]:
df = pd.read_csv("/content/drive/MyDrive/t5_multiTaskBaseModelQG_DataFrame.csv")

In [31]:
df

,Unnamed: 0,Questions,Answers
0,0,What are the layers that make up the convolutional neural network?,pooling layers
1,1,What type of deep learning defined multi neural network architecture contain...,deep learning
2,2,How many parts is artificial intelligence divided into?,three
3,3,What kind of system stimulates the way human brain analyzes information?,artificial neural network computing system
4,4,What model creates general intelligence?,human brain
5,5,What kind of system stimulates the way human brain analyzes information?,artificial neural network computing system
6,6,What is supervised learning?,best suited problems set available reference points ground truth train algor...
7,7,What is convolutional neural network hidden layers?,layers architecture
8,8,Watson image facial recognition software disease mapping prediction tools?,ibm


#**Answering Questions with haystack**

In [36]:
df['Questions'][0]


'What are the layers that make up the convolutional neural network?'

In [33]:
prediction = pipe.run(
    query=df['Questions'][0], params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

Inferencing Samples: 100%|██████████| 1/1 [00:01<00:00,  1.05s/ Batches]


In [34]:
print(prediction)

{'query': 'What are the layers that make up the convolutional neural network?', 'no_ans_gap': 8.421114921569824, 'answers': [<Answer {'answer': 'convolutional layers , pooling layers , fully connected layers , normalization layers', 'type': 'extractive', 'score': 0.7687554359436035, 'context': ' hidden layers convolutional neural network  convnet  consists convolutional layers , pooling layers , fully connected layers , normalization layers .', 'offsets_in_document': [{'start': 111, 'end': 196}], 'offsets_in_context': [{'start': 63, 'end': 148}], 'document_id': '252746d265d48451db9759c74f7c69ea', 'meta': {}}>, <Answer {'answer': 'convolutional layers , pooling layers , fully connected layers , normalization layers', 'type': 'extractive', 'score': 0.7096093893051147, 'context': ' hidden layers convolutional neural network  convnet  consists convolutional layers , pooling layers , fully connected layers , normalization layers .', 'offsets_in_document': [{'start': 119, 'end': 204}], 'offs

#**Evaluation**

**The Question is : What are the layers that make up the convolutional neural network? <br>
T5 answer: Pooling layers<br>
haystack answer: convolutional layers , pooling layers , fully connected layers , normalization<br> 
it is clear that hystack answers is more accurate**